In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from util import *

In [2]:
data = pd.read_csv('../../input/data.csv')

In [3]:
# 数据集2：annual_report_info.csv
# 包含数据集7和8中涉及到的企业的年报基本信息，每一行代表一个企业的年报基本数据，每一行有23列，其中id列为企业唯一标识，列之间采用“,”分隔符分割。
# 数据格式如下：
# [id:企业唯一标识, ANCHEYEAR:年度, STATE:状态, FUNDAM:资金数额, MEMNUM:成员人数, FARNUM:农民人数, ANNNEWMEMNUM:本年度新增成员人数, ANNREDMEMNUM:本年度退出成员人数, EMPNUM:从业人数, EMPNUMSIGN:从业人数是否公示, BUSSTNAME:经营状态名称, COLGRANUM:其中高校毕业生人数经营者, RETSOLNUM:其中退役士兵人数经营者, DISPERNUM:其中残疾人人数经营者, UNENUM:其中下岗失业人数经营者, COLEMPLNUM:其中高校毕业生人数雇员, RETEMPLNUM:其中退役士兵人数雇员, DISEMPLNUM:其中残疾人人数雇员, UNEEMPLNUM:其中下岗失业人数雇员, WEBSITSIGN:是否有网站标志, FORINVESTSIGN:是否有对外投资企业标志, STOCKTRANSIGN:有限责任公司本年度是否发生股东股权转让标志, PUBSTATE:公示状态：1 全部公示，2部分公示,3全部不公示]
annual_report_info = pd.read_csv('../../input/train/annual_report_info.csv')
print(annual_report_info.shape)
annual_report_info.info()

(22550, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22550 entries, 0 to 22549
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             22550 non-null  object 
 1   ANCHEYEAR      22550 non-null  float64
 2   STATE          22545 non-null  float64
 3   FUNDAM         5702 non-null   float64
 4   MEMNUM         29 non-null     float64
 5   FARNUM         29 non-null     float64
 6   ANNNEWMEMNUM   29 non-null     float64
 7   ANNREDMEMNUM   29 non-null     float64
 8   EMPNUM         22535 non-null  float64
 9   EMPNUMSIGN     16833 non-null  float64
 10  BUSSTNAME      17680 non-null  object 
 11  COLGRANUM      20041 non-null  float64
 12  RETSOLNUM      20041 non-null  float64
 13  DISPERNUM      20041 non-null  float64
 14  UNENUM         20041 non-null  float64
 15  COLEMPLNUM     20041 non-null  float64
 16  RETEMPLNUM     20041 non-null  float64
 17  DISEMPLNUM     20041 non-null  float64

In [4]:
annual_report_info.head()

,id,ANCHEYEAR,STATE,FUNDAM,MEMNUM,FARNUM,ANNNEWMEMNUM,ANNREDMEMNUM,EMPNUM,EMPNUMSIGN,...,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2017.0,2.0,5.0,NaN,NaN,NaN,NaN,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2018.0,2.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2017.0,2.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2018.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
4,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2017.0,2.0,5.0,NaN,NaN,NaN,NaN,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0


In [5]:
def identify_missing(df, missing_threshold):
    """
    缺失率
    @param df:
    @param missing_threshold:
    @return:
    """
    missing_rate = df.isnull().sum() / len(df)
    missing_rate = missing_rate.sort_values(ascending=False)
    print(missing_rate)
    to_drop = missing_rate[missing_rate > missing_threshold].index.to_list()
    print('{} features with greater than {} missing values.\n'.format(
        len(to_drop), missing_threshold))
    return to_drop

In [6]:
to_drop = identify_missing(annual_report_info, missing_threshold=0.9)
annual_report_info.drop(to_drop, axis=1, inplace=True)
to_drop

FARNUM           0.998714
ANNNEWMEMNUM     0.998714
ANNREDMEMNUM     0.998714
MEMNUM           0.998714
FUNDAM           0.747140
STOCKTRANSIGN    0.401020
FORINVESTSIGN    0.268780
EMPNUMSIGN       0.253525
BUSSTNAME        0.215965
RETSOLNUM        0.111264
COLGRANUM        0.111264
UNENUM           0.111264
COLEMPLNUM       0.111264
RETEMPLNUM       0.111264
DISEMPLNUM       0.111264
UNEEMPLNUM       0.111264
DISPERNUM        0.111264
WEBSITSIGN       0.001463
PUBSTATE         0.000887
EMPNUM           0.000665
STATE            0.000222
ANCHEYEAR        0.000000
id               0.000000
dtype: float64
4 features with greater than 0.9 missing values.



['FARNUM', 'ANNNEWMEMNUM', 'ANNREDMEMNUM', 'MEMNUM']

In [7]:
annual_report_info['id'].nunique()

8937

In [8]:
annual_report_info.head()

,id,ANCHEYEAR,STATE,FUNDAM,EMPNUM,EMPNUMSIGN,BUSSTNAME,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2017.0,2.0,5.0,10.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2018.0,2.0,2.0,2.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2017.0,2.0,NaN,4.0,2.0,开业,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2018.0,2.0,NaN,3.0,2.0,开业,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
4,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2017.0,2.0,5.0,10.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0


In [9]:
annual_report_info.nunique()

id               8937
ANCHEYEAR           4
STATE               2
FUNDAM            167
EMPNUM            270
EMPNUMSIGN          2
BUSSTNAME           4
COLGRANUM          43
RETSOLNUM          14
DISPERNUM           5
UNENUM             35
COLEMPLNUM         94
RETEMPLNUM         26
DISEMPLNUM         16
UNEEMPLNUM         72
WEBSITSIGN          2
FORINVESTSIGN       2
STOCKTRANSIGN       2
PUBSTATE            3
dtype: int64